# Hyperparameter Tuning

This notebook demonstrates how to automatically find optimal aggregation parameters using `find_optimal_combination`.

Instead of manually choosing the number of periods and segments, you can specify a target data reduction and let tsam find the best combination.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import plotly.express as px
import plotly.io as pio

import tsam
from tsam.tuning import find_optimal_combination

pio.renderers.default = "notebook"

### Load test data

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0, parse_dates=True)
print(f"Shape: {raw.shape}")
print(f"Timesteps: {len(raw)}")
raw.head()

### Find optimal combination for 2% data reduction

The `find_optimal_combination` function searches for the best period/segment combination that achieves the target data reduction while minimizing RMSE.

Use `n_jobs=-1` to utilize all available CPUs for faster search.

In [ ]:
result = find_optimal_combination(
    raw,
    data_reduction=0.02,  # Target: 2% of original timesteps
    period_duration=24,
    n_jobs=-1,  # Use all CPUs
    show_progress=True,
)

print("\nOptimal configuration:")
print(f"  Periods: {result.n_clusters}")
print(f"  Segments: {result.n_segments}")
print(f"  RMSE: {result.rmse:.4f}")
print(f"  Timesteps: {result.n_clusters * result.n_segments}")
print(f"  Reduction: {result.n_clusters * result.n_segments / len(raw) * 100:.2f}%")

### View the search history

The tuning result includes the history of all tested configurations.

In [ ]:
history_df = pd.DataFrame(result.history)
history_df.sort_values("rmse")

### Use the optimal result

The `best_result` attribute contains the full `AggregationResult` for the optimal configuration.

In [ ]:
best = result.best_result

print(f"Typical periods shape: {best.cluster_representatives.shape}")
print("\nAccuracy metrics:")
print(best.accuracy)

In [ ]:
reconstructed = best.reconstruct()

unstacked = tsam.unstack_to_periods(reconstructed, period_duration=24)
px.imshow(
    unstacked["T"].values.T,
    labels={"x": "Day", "y": "Hour", "color": "Temperature"},
    title=f"Optimal: {result.n_clusters} periods x {result.n_segments} segments",
    aspect="auto",
)

### Compare different reduction targets

Let's see how the optimal configuration changes with different data reduction targets.

In [ ]:
reductions = [0.01, 0.02, 0.05, 0.10]
results_comparison = {}

for reduction in reductions:
    r = find_optimal_combination(
        raw,
        data_reduction=reduction,
        period_duration=24,
        n_jobs=-1,
        show_progress=False,
    )
    results_comparison[f"{int(reduction * 100)}%"] = r
    print(
        f"{int(reduction * 100)}% reduction: {r.n_clusters} periods x {r.n_segments} segments, RMSE={r.rmse:.4f}"
    )

In [ ]:
comparison_data = {"Original": raw}
for label, r in results_comparison.items():
    comparison_data[label] = r.best_result.reconstruct()

# Duration curve comparison using plotly express
frames = []
for name, df in comparison_data.items():
    sorted_vals = df["Load"].sort_values(ascending=False).reset_index(drop=True)
    frames.append(
        pd.DataFrame(
            {"Hour": range(len(sorted_vals)), "Load": sorted_vals, "Method": name}
        )
    )
long_df = pd.concat(frames, ignore_index=True)

px.line(
    long_df,
    x="Hour",
    y="Load",
    color="Method",
    title="Duration Curve Comparison - Different Reduction Targets",
)

### Helper functions

Use `find_clusters_for_reduction` and `find_segments_for_reduction` to calculate parameters for a specific reduction target.

In [ ]:
from tsam.tuning import find_clusters_for_reduction, find_segments_for_reduction

n_timesteps = len(raw)
target_reduction = 0.01  # 1% of original

# How many periods can we have with 24 segments?
max_periods = find_clusters_for_reduction(
    n_timesteps, n_segments=24, data_reduction=target_reduction
)
print(f"With 24 segments: max {max_periods} periods for 1% reduction")

# How many segments can we have with 8 periods?
max_segments = find_segments_for_reduction(
    n_timesteps, n_clusters=8, data_reduction=target_reduction
)
print(f"With 8 periods: max {max_segments} segments for 1% reduction")